In [0]:
import numpy as np
import cv2
import os

In [0]:
path = "C:\\Users\\Cordu\\traffic signs\\"
images = np.empty([len(os.listdir(path)),64,64,3])
i = 0

for file in os.listdir(path):
    image = cv2.imread(path + file)
    img = cv2.resize(image,(64,64))
    im_array = np.asarray(img) / 255
    images[i] = im_array
    i = i + 1

In [3]:
import matplotlib.pyplot as plt

im = images[542]
print(im.shape)
plt.imshow(im)

(64, 64, 3)


In [4]:
import keras
from keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten, Input, Activation
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Conv2DTranspose, GaussianNoise, InputLayer

Using TensorFlow backend.


In [5]:
def create_generator():
    epsilon = 0.00001 # Small float added to variance to avoid dividing by zero in the BatchNorm layers.
    noise_shape = (100,)

    model = Sequential()

    model.add(Dense(4*4*512, activation='linear', input_shape=(100,)))
    model.add(Reshape((4, 4, 512)))

    model.add(Conv2DTranspose(256, kernel_size=[5,5], strides=[2,2], padding="same",
                              kernel_initializer= keras.initializers.TruncatedNormal(stddev=0.02)))
    model.add(BatchNormalization(momentum=0.9, epsilon=epsilon))
    model.add(Activation('relu'))

    model.add(Conv2DTranspose(256, kernel_size=[5,5], strides=[2,2], padding="same",
                              kernel_initializer= keras.initializers.TruncatedNormal(stddev=0.02)))
    model.add(BatchNormalization(momentum=0.9, epsilon=epsilon))
    model.add(Activation('relu'))

    model.add(Conv2DTranspose(128, kernel_size=[5,5], strides=[2,2], padding="same",
                              kernel_initializer= keras.initializers.TruncatedNormal(stddev=0.02)))
    model.add(BatchNormalization(momentum=0.9, epsilon=epsilon))
    model.add(Activation('relu'))

    model.add(Conv2DTranspose(64, kernel_size=[5,5], strides=[2,2], padding="same",
                              kernel_initializer= keras.initializers.TruncatedNormal(stddev=0.02)))
    model.add(BatchNormalization(momentum=0.9, epsilon=epsilon))
    model.add(Activation('relu'))

    model.add(Conv2DTranspose(3, kernel_size=[5,5], strides=[1,1], padding="same",
                              kernel_initializer= keras.initializers.TruncatedNormal(stddev=0.02)))
    # Standard activation for the generator of a GAN
    model.add(Activation("tanh"))

    model.summary()

    noise = Input(shape=(100,))
    img = model(noise)

    return Model(noise, img)

generator = create_generator()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8192)              827392    
_________________________________________________________________
reshape_1 (Reshape)          (None, 4, 4, 512)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 8, 8, 256)         3277056   
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 256)         1024      
_________________________________________________________________
activation_1 (Activation)    (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 16, 16, 256)       1638656   
_________________________________________________________________
batc

In [6]:
def build_discriminator():

    model = Sequential()

    model = Sequential()
    model.add(InputLayer(input_shape=(64,64,3)))
    model.add(GaussianNoise(0.2))
    model.add(Conv2D(32, (3, 3), strides=[2,2],padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization())
    model.add(Dropout(.3))
    model.add(Conv2D(64, (3, 3),strides=[2,2], kernel_initializer='he_uniform', padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization())
    model.add(Dropout(.3))
    model.add(Conv2D(128, (3, 3),strides=[2,2], kernel_initializer='he_uniform', padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization())
    model.add(Dropout(.3))
    model.add(Conv2D(256, (3, 3),strides=[2,2], kernel_initializer='he_uniform', padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization())
    model.add(Dropout(.3))
    model.add(Conv2D(512, (3, 3),strides=[2,2], kernel_initializer='he_uniform', padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization())
    model.add(Dropout(.3))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
        
    model.summary()
        
    img = Input(shape=(64,64,3))
    validity = model(img)

    return Model(img, validity)

discriminator = build_discriminator()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gaussian_noise_1 (GaussianNo (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
___________________________

In [7]:
def get_gan():
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    model.summary()
    
    return model

gan = get_gan()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 64, 64, 3)         6774915   
_________________________________________________________________
model_2 (Model)              (None, 1)                 1574593   
Total params: 8,349,508
Trainable params: 8,346,116
Non-trainable params: 3,392
_________________________________________________________________


In [0]:
optimizer = Adam(0.0002,0.5)
discriminator.compile(loss='binary_crossentropy', 
                                   optimizer=optimizer,
                                   metrics=['accuracy'])
discriminator.trainable = False
generator.compile(loss='binary_crossentropy', optimizer=optimizer)
gan.compile(loss='binary_crossentropy', optimizer=optimizer)

In [15]:
from keras.models import load_model
discriminator = load_model('traffic_discriminator.h5')
generator = load_model('traffic_generator.h5')

C:\Users\Cordu\Anaconda3\lib\site-packages\keras\engine\saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [0]:
from PIL import Image

def save_images(epoch):
        noise = np.random.normal(0, 1, [100, 100])
        image_array = np.empty([25,64,64,3])
        gen_imgs = generator.predict(noise)
        image_count = 0
        for i in range(5):
                image_array[i] = gen_imgs[i]
                im = np.asarray(image_array[i] * 255)
                cv2.imwrite("C:\\Users\\Cordu\\trafficSigns2\\image_%d.png" % (epoch + i) ,im)

In [0]:
from keras.models import load_model

epochs = 100000
d_loss = [0,0]
g_loss = 0
mean_d_loss=[0,0]
mean_g_loss=0

def train_stuff():
    d_loss = [0,0]
    g_loss = 0
    for i in range(epochs):
        noise= np.random.normal(0,1, [64, 100])
        fake_images = generator.predict(noise)
        fake_output = np.full((64, 1), 0.9)
        indexes = np.random.randint(0, len(os.listdir(path)), 64)
        real_images = images[indexes]
        real_output = np.ones((64,1))
        d_loss = np.add(d_loss, discriminator.train_on_batch(fake_images, fake_output))
        d_loss = np.add(d_loss, discriminator.train_on_batch(real_images, real_output))
        d_loss = d_loss * 0.5
        noise= np.random.normal(0,1, [128, 100])
        g_loss += gan.train_on_batch(noise,np.full((128, 1), 1))
        #g_loss += gan.train_on_batch(noise,np.full((128, 1), 1))
        
        if i % 100 == 0:
            print(d_loss)
            print(g_loss)
            d_loss = [0,0]
            g_loss = 0
            save_images(i)
        if i % 1000 == 0:
            discriminator.save('traffic_discriminator.h5')  # creates a HDF5 file 'my_model.h5'
            generator.save('traffic_generator.h5')

                           
train_stuff()

[0.17035612 0.5       ]
0.07815942168235779
[0.33682042 1.        ]
6.099033806473017
[0.33886766 1.        ]
5.3131190445274115
[0.34342051 1.        ]
4.50166585482657
[0.34292802 1.        ]
4.338348468765616
[0.33998114 1.        ]
3.917978683486581
[0.33958762 1.        ]
3.592452509328723
[0.33827457 1.        ]
3.336326526477933
[0.33809267 1.        ]
3.051804857328534
[0.3405548 1.       ]
2.8630552031099796
[0.3549954 1.       ]
2.4260390289127827
[0.3399342 1.       ]
2.193451332859695
[0.33610534 1.        ]
1.922171564772725
[0.3332523 1.       ]
1.6771135870367289
[0.34610835 1.        ]
1.5498236482962966
[0.33159786 1.        ]
1.4178434247151017
[0.33227175 1.        ]
1.276349431835115
[0.34106925 1.        ]
1.115956143476069
[0.34091549 1.        ]
1.0503346603363752
[0.34163408 1.        ]
0.9602141892537475
[0.33155591 1.        ]
0.8803793219849467
[0.3352328 1.       ]
0.816393066663295
[0.33421599 1.        ]
0.7168760960921645
[0.33207985 1.        ]
0.6585962